The goal of this notebook is to walk through an example use-case for the reweighted loss function. My goal is to give sufficient background information that those unfamilar with NLI can still follow along and understand the rationale for and results of changing the loss function. The work in this notebook was originally done for a final paper for a graduate class in NLP, but I wanted to repackage the content of the paper in a more accessable format, resulting in this notebook. After each section I'll add some links for the sources of each section and more relevant backgorund material for those wanting to go deeper into the topic. 

This notebook does not contain much code. If you would rather just see the huggingface and pytorch code used to train and evaluate the model see './python/run.py' and './python/reweighted_training.py'. For an example of how to use these files code see './notebook/example.ipynb'.

### Overview
The goal of this projects was to explore data artifacts found in one common dataset used to evaluate LLMs and try some techniques to mitigate their effect. Specifically, this will focus on the task of natural language inference (NLI) and the Stanford NLI (SNLI) dataset. This dataset has been shown to contian multiple artifacts but I will focus on 3 syntactic heuristics (described later). I attempted to use two different techniques to reduce the impact of the dataset artifacts: adding training data, and adapting the training loss function. Adding training data preforms well but is not an ideal solution and is mostly used to show the performance shortcomings of adapting the loss function.

### NLI and SNLI
NLI (Natural Language Inference) is the task of determining if a premise sentence entails (supports), is neutral to, or contradicts a hypothesis sentence. It has been used recently as one way of evaluating if LLMs are able to understand the contents of a sentence. One widely used benchmark dataset for this task is SNLI, which contains 500k+ sentence pairs. An example sentence pair from SNLI is:

premise: A baby at the end of a slip and slide at a party.  \
hypothesis: The baby is wet. \
label: contradiction

In this example the correct output label is entailment because going down a slip and slide results in getting wet. To keep this document reasonably short I won't include entailment and neutral examples but these can be found at the SNLI dataset link below.

Sources + Additional reading: \
NLI paper: https://aclanthology.org/C08-1066/ \
SNLI paper: https://nlp.stanford.edu/pubs/snli_paper.pdf \
SNLI dataset: https://nlp.stanford.edu/projects/snli/

### SNLI Artifacts
An artifact in a dataset is a feature of the dataset that unintentionally correlates with one of the output labels. LLMs trained on datasets with artifacts can be described as being biased. LLMs can use artifacts to improve their performance when making predictions because it is often far easier to detect artifacts than it is to truely understand the contents of a sentence. This is not ideal because it allows models to avoid having to deeply understand sentences to make predictions and can inflate model performance. Additionally, different datasets often don't have the same artifacts, so a model trained on one dataset with artifacts in it will not perform well when evaluated on another dataset. 

The SNLI dataset contains multiple types of artifacts. One simple example is that hypothesis sentences containing the word 'not' correlate to the output label of contradiction. The specific set of artifacts this project focuses on are all to do with the hypothesis and premise sentences containing a set of the same words. The reason this is an artifact is that the majority of sentences in SNLI that contain a set of the same set of words have an output label of entailment. An example of this is the sentence pair:

premise: A big brown dog swims towards the camera \
hypothesis: A dog swims towards the camera \
label: entailment

Examples like the one above can further be broken down into 3 categories: lexical overlaps, constituents, and subsequences (the details of these categories are described more in the HANS paper).  
![Table 1](./img/table1.png) \
Table 1 gives a count of each of these artifact categories by output label in the SNLI dataset. As the majority of examples in these categories have an output label of entailment, a model trained on the SNLI dataset will associate the presence of these categories with the entailment label. While this works well most of the time, when one of these categories is present and the output label is not entailment, the model can struggle. 

This might not be a huge issue on the SNLI dataset, but for other datasets it can be. One such dataset is the HANS dataset which was specifically created to highlight this issue. Unlike SNLI, the output labels are balanced for examples that fall into the 3 categories, being evenly split across the entailment and contradiction labels. Models trained on the SNLI dataset have very poor performance when evaluated on HANS often struggling to acheive much over a 50% accuracy. 

Sources + Additional reading: \
SNLI artifacts paper: https://aclanthology.org/C18-1198/ \
HANS paper: https://arxiv.org/abs/1902.01007 \
HANS dataset + repository: https://github.com/tommccoy1/hans

### Base Model
The model used for the remainder of this notebook will be the ELECTRA-small model. There are many other models that will work in place of this one. I chose the ELECTRA-small model because it works reasonably well and doesn't take too long to train compared to larger models. 

The base model simply refers to the ELECTRA-small model trained on the SNLI dataset with no additional corrective measures. The following sections will look at ways to improve (debias) the base model.

Sources + Additional reading: \
ELECTRA paper: https://arxiv.org/abs/2003.10555

### Improving the Base Model
#### Adding Training Data
One straightforward way to debias a model is to simply add more training data to counter the artifact in the dataset. For instance, in our example one could add sententence pairs that contain the same set of words but have an output label of neutral or contradiction.  If enough examples are added, sentence pairs having the same set of words would no longer correlate with the output label of entailment and the artifact would be gone. This can be a very effective way to remove artifacts given a knowledge of what the artifacts in the data are, and a means of creating potentially thousands of new training examples. 

Fortunately, for this particular example the creators of the HANS evlaution dataset described in the last section also created a training dataset too. Training your model on SNLI + the HANS trainining dataset can result in a debiased model. In the results section this process will be refered to as annealing. 

#### Reweighted loss
To address 

### Results